In [3]:
from gensim import corpora,models,similarities

documents = ["Human machine interface for lab abc computer applications",
             "A survey of user opinion of computer system response time",
             "The EPS user interface management system",
             "System and human system engineering testing of EPS",
             "Relation of user perceived response time to error measurement",
             "The generation of random binary unordered trees",
             "The intersection graph of paths in trees",
             "Graph minors IV Widths of trees and well quasi ordering",
             "Graph minors A survey"]

In [4]:
stoplist = set('for a of the end and to in'.split())

texts = [[word for word in document.lower().split() if word not in stoplist] for document in documents]

In [6]:
from collections import defaultdict
frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [7]:
from pprint import pprint
pprint(texts)

[['human', 'interface', 'computer'],
 ['survey', 'user', 'computer', 'system', 'response', 'time'],
 ['eps', 'user', 'interface', 'system'],
 ['system', 'human', 'system', 'eps'],
 ['user', 'response', 'time'],
 ['trees'],
 ['graph', 'trees'],
 ['graph', 'minors', 'trees'],
 ['graph', 'minors', 'survey']]


In [8]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(12 unique tokens: [u'minors', u'graph', u'system', u'trees', u'eps']...)


In [10]:
print(dictionary.token2id)

{u'minors': 11, u'graph': 10, u'system': 6, u'trees': 9, u'eps': 8, u'computer': 1, u'survey': 5, u'user': 7, u'human': 2, u'time': 4, u'interface': 0, u'response': 3}


In [11]:
corpus = [dictionary.doc2bow(text) for text in texts]
pprint(corpus)

[[(0, 1), (1, 1), (2, 1)],
 [(1, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1)],
 [(0, 1), (6, 1), (7, 1), (8, 1)],
 [(2, 1), (6, 2), (8, 1)],
 [(3, 1), (4, 1), (7, 1)],
 [(9, 1)],
 [(9, 1), (10, 1)],
 [(9, 1), (10, 1), (11, 1)],
 [(5, 1), (10, 1), (11, 1)]]


In [12]:
from gensim import matutils
numpy_matrix = matutils.corpus2dense(corpus,num_terms=12)
print(numpy_matrix)

[[ 1.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 1.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  1.]
 [ 0.  1.  1.  2.  0.  0.  0.  0.  0.]
 [ 0.  1.  1.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  1.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  1.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.]]


In [13]:
scipy_csc_matrix = matutils.corpus2csc(corpus)
numpy_corpus = matutils.Dense2Corpus(numpy_matrix)
scipy_corpus = matutils.Sparse2Corpus(scipy_csc_matrix)

In [14]:
#TFIDF

tfidf = models.TfidfModel(corpus)


In [15]:
corpus_tfidf = tfidf[corpus]

for doc in corpus_tfidf:
    print(doc)

[(0, 0.5773502691896257), (1, 0.5773502691896257), (2, 0.5773502691896257)]
[(1, 0.44424552527467476), (3, 0.44424552527467476), (4, 0.44424552527467476), (5, 0.44424552527467476), (6, 0.3244870206138555), (7, 0.3244870206138555)]
[(0, 0.5710059809418182), (6, 0.4170757362022777), (7, 0.4170757362022777), (8, 0.5710059809418182)]
[(2, 0.49182558987264147), (6, 0.7184811607083769), (8, 0.49182558987264147)]
[(3, 0.6282580468670046), (4, 0.6282580468670046), (7, 0.45889394536615247)]
[(9, 1.0)]
[(9, 0.7071067811865475), (10, 0.7071067811865475)]
[(9, 0.5080429008916749), (10, 0.5080429008916749), (11, 0.695546419520037)]
[(5, 0.6282580468670046), (10, 0.45889394536615247), (11, 0.6282580468670046)]


In [16]:
#LSI transformation

lsi = models.LsiModel(corpus_tfidf,id2word=dictionary,num_topics=2)
lsi.print_topics(2)

[u'0.703*"trees" + 0.538*"graph" + 0.402*"minors" + 0.187*"survey" + 0.061*"system" + 0.060*"response" + 0.060*"time" + 0.058*"user" + 0.049*"computer" + 0.035*"interface"',
 u'0.460*"system" + 0.373*"user" + 0.332*"eps" + 0.328*"interface" + 0.320*"response" + 0.320*"time" + 0.293*"computer" + 0.280*"human" + 0.171*"survey" + -0.161*"trees"']

In [17]:
#bow -> tfidf -> lsi

corpus_lsi = lsi[corpus_tfidf]
for doc in corpus_lsi:
    print(doc)

[(0, 0.06600783396090551), (1, 0.5200703306361848)]
[(0, 0.19667592859142691), (1, 0.76095631677000375)]
[(0, 0.089926399724466047), (1, 0.7241860626752501)]
[(0, 0.075858476521783402), (1, 0.63205515860034223)]
[(0, 0.10150299184980256), (1, 0.57373084830029497)]
[(0, 0.70321089393783087), (1, -0.16115180214025981)]
[(0, 0.87747876731198271), (1, -0.16758906864659651)]
[(0, 0.90986246868185716), (1, -0.14086553628719267)]
[(0, 0.61658253505692773), (1, 0.053929075663891893)]


In [19]:
#random projection model

rp = models.RpModel(corpus_tfidf, num_topics=500)

rp

In [20]:
?rp

In [23]:
lda = models.LdaModel(corpus, id2word=dictionary, num_topics=100)

In [24]:
lda

In [25]:
?lda

In [27]:
lda.print_topics(2)

[u'0.083*user + 0.083*survey + 0.083*graph + 0.083*trees + 0.083*eps + 0.083*interface + 0.083*system + 0.083*human + 0.083*response + 0.083*computer',
 u'0.083*user + 0.083*survey + 0.083*graph + 0.083*trees + 0.083*eps + 0.083*interface + 0.083*system + 0.083*human + 0.083*response + 0.083*computer']

In [28]:
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow] # convert the query to LSI space
print(vec_lsi)

[(0, 0.079104751174451024), (1, 0.57328352430794016)]


In [30]:
index = similarities.MatrixSimilarity(corpus_lsi)

In [31]:
sims = index[vec_lsi]
print(list(enumerate(sims)))

[(0, 0.99994081), (1, 0.99330217), (2, 0.99990785), (3, 0.99984384), (4, 0.9992786), (5, -0.08804217), (6, -0.0515742), (7, -0.016480923), (8, 0.22248439)]


In [34]:
sims = sorted(enumerate(sims), key = lambda item : -item[1])
pprint(sims)

[(0, 0.99994081),
 (2, 0.99990785),
 (3, 0.99984384),
 (4, 0.9992786),
 (1, 0.99330217),
 (8, 0.22248439),
 (7, -0.016480923),
 (6, -0.0515742),
 (5, -0.08804217)]
